In [1]:
import pyarrow.parquet as pq
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#parquet_file = pq.ParquetFile('/eos/experiment/wcte/wcte_tests/mPMT_periodic_readout/2024-11-24/periodic_readout_20241124210744_2_waveforms.parquet')
parquet_file = pq.ParquetFile('/eos/experiment/wcte/wcte_tests/mPMT_led_events/2024-11-27/20241127214613/led_scan_card_77_20241127214613_0_waveforms.parquet')

batch_size = 10000
batch_id = 0

pulses_nbins = 280
pulses_width = 0.5
nPMTs_max = 2000
dt = 8
fDigiTimeOffset = -30.1

pmt_dict = {}

In [2]:
amplitude = []
digiQ = []
digiT = []
amplitude_pmt = [[] for j in range(nPMTs_max)]
pulses = [[[] for i in range(pulses_nbins)] for j in range(nPMTs_max)]

for batch in parquet_file.iter_batches(batch_size=batch_size):
    print("batch ",batch_id)
    batch_id += 1
    v = batch.to_pandas()
    v = v[v["card_id"]<130]
    waveforms = np.array(list(v["samples"].values))
    card_id = np.array(list(v["card_id"].values))
    chan = np.array(list(v["chan"].values))
    #res=np.array(list(v["samples"].values)).max(axis=1)
    res_idx=waveforms.argmax(axis=1)
    res=np.take_along_axis(waveforms,np.expand_dims(res_idx,axis=1),axis=1).squeeze(axis=1)
    amplitude.extend(list(res))
    for i in range(len(res_idx)):
        pmt_id = -1
        key = (card_id[i],chan[i])
        if key in pmt_dict:
            pmt_id = pmt_dict[key]
        else:
            pmt_id = len(pmt_dict)
            pmt_dict[key] = pmt_id
        amplitude_pmt[pmt_id].append(res[i])
        if waveforms[i][res_idx[i]]>20 and res_idx[i]>=5 and res_idx[i]<=251 and np.sum(waveforms[i][res_idx[i]-2:res_idx[i]+5])>40:
            charge = 0
            if waveforms[i][res_idx[i]+2]>0:
                charge = np.sum(waveforms[i][res_idx[i]-5:res_idx[i]+3])
            else:
                charge = np.sum(waveforms[i][res_idx[i]-5:res_idx[i]+2])
            val1 = waveforms[i][res_idx[i]] - waveforms[i][res_idx[i]-1]
            val2 = waveforms[i][res_idx[i]+1] - waveforms[i][res_idx[i]]
            digiT.append(dt*val1/(val1-val2)+fDigiTimeOffset)
            digiQ.append(charge/135.556640625)
            time = dt*(res_idx[i]+val1/(val1-val2))+fDigiTimeOffset
            #print("time = ",time)
            for j in range(res_idx[i]-3,min(res_idx[i]+18,256)):
            #for j in range(res_idx[i]-3,res_idx[i]+3):
                idx = int((j*dt-time)/pulses_width)
                if idx>=0 and idx<pulses_nbins:
                    val = waveforms[i][j]/charge
                    if val>-1 and val<1:
                        pulses[pmt_id][idx].append(val)
                        # if idx>0:
                        #     pulses[pmt_id][idx-1].append(val)
                        # if idx+1<pulses_nbins:
                        #     pulses[pmt_id][idx+1].append(val)
    # if batch_id > 10:
    #     break
    # data_dict = batch.to_pydict()
    # for i in range(len(data_dict['samples'])):
    #     amplitude.append(max(data_dict['samples'][i]))

batch  0
batch  1
batch  2
batch  3
batch  4
batch  5
batch  6
batch  7
batch  8
batch  9
batch  10
batch  11
batch  12
batch  13
batch  14
batch  15
batch  16
batch  17
batch  18
batch  19
batch  20
batch  21
batch  22
batch  23
batch  24
batch  25
batch  26
batch  27
batch  28
batch  29
batch  30
batch  31
batch  32
batch  33
batch  34
batch  35
batch  36
batch  37
batch  38
batch  39
batch  40
batch  41
batch  42
batch  43
batch  44
batch  45
batch  46
batch  47
batch  48
batch  49
batch  50
batch  51
batch  52
batch  53
batch  54
batch  55
batch  56
batch  57
batch  58
batch  59
batch  60
batch  61
batch  62
batch  63
batch  64
batch  65
batch  66
batch  67
batch  68
batch  69
batch  70
batch  71
batch  72
batch  73
batch  74
batch  75
batch  76
batch  77
batch  78
batch  79
batch  80
batch  81
batch  82
batch  83
batch  84
batch  85
batch  86
batch  87
batch  88
batch  89
batch  90
batch  91
batch  92
batch  93
batch  94
batch  95
batch  96
batch  97
batch  98
batch  99
batch  100

In [ ]:
len(amplitude)
n, bins, patches = plt.hist(amplitude,bins=range(5,200))
print(n)
print(bins)
plt.xticks(np.linspace(10, 200, 20))
plt.grid(visible=True)
plt.savefig("fig/20241127214613/amplitude.pdf")
plt.clf()

[12652. 12012.  6160.  6085. 14549.  6621.  4909.  4546.  4457.  4366.
  9816.  4246.  4435.  4400.  4432.  4438.  4931.  5273.  5433.  5581.
  5782.  6216.  7007.  6561.  5638.  5931.  6085.  6246.  6502.  6742.
  6897.  7289.  7522.  7742.  8027.  8435.  8725.  8915.  9155.  9523.
  9712. 10011. 10479. 10658. 10773. 11273. 11492. 11587. 11860. 12099.
 12326. 12367. 12492. 12693. 12723. 13121. 12975. 12981. 13378. 19058.
 13559. 13506. 13405. 13403. 13564. 13210. 13250. 13137. 13185. 13268.
 13080. 12986. 13027. 13021. 12669. 12649. 12494. 12198. 12326. 12213.
 11793. 11859. 11565. 11612. 11375. 11057. 11009. 11062. 10885. 10561.
 10200. 10218. 10070.  9509.  9556.  9354.  9339.  8908.  8965.  8683.
  8435.  8456.  8333.  7977.  7806.  7721.  7240.  7321.  7069.  6963.
  6795.  6644.  6436.  6113.  6097.  5837.  5768.  5429.  5419.  5350.
  5124.  4899.  4802.  4799.  4645.  4692.  4391.  4277.  3952.  3969.
  3785.  3806.  3629.  3604.  3406.  3345.  3246.  3235.  3131.  3034.
  2950

<Figure size 640x480 with 0 Axes>

In [3]:
from pylab import *
from scipy.optimize import leastsq
from scipy.optimize import minimize

fitfunc  = lambda p, x: p[0]*(exp(-0.5*((x-p[1])/p[2])**2)+p[5]/2/sqrt(2)*exp(-0.5*0.5*((x-2*p[1])/p[2])**2)+p[5]**2/2/3/sqrt(3)*exp(-0.5/3*((x-3*p[1])/p[2])**2))+p[3]*exp(-p[4]*x)
errfunc_leastsq  = lambda p, x, y: (y - fitfunc(p, x))

gains = [-1 for j in range(nPMTs_max)]
sigmas = [-1 for j in range(nPMTs_max)]
mu = [-1 for j in range(nPMTs_max)]
alpha = [-1 for j in range(nPMTs_max)]

for (card_id, chan), value in pmt_dict.items():
    if len(amplitude_pmt[value])>100:
        n, bins, patches = plt.hist(amplitude_pmt[value],bins=39,range=[5,200])
        if max(n) > 1:
            bins=bins[:-1]
            plt.xticks(np.linspace(20, 200, 10))
            plt.grid(visible=True)
            init  = [max(n), 60, 30, 10, 0.05, 0.206]
            out   = leastsq( errfunc_leastsq, init, args=(bins, n))
            c_leastsq=out[0]
            print(f"fig/20241127214613/{card_id}_{chan}_amplitude.pdf, mean={c_leastsq[1]}, sigma={c_leastsq[2]}, mu={c_leastsq[5]}")
            gains[value] = (c_leastsq[1])
            sigmas[value] = (c_leastsq[2]/c_leastsq[1])
            mu[value] = c_leastsq[5]
            alpha[value] = c_leastsq[4]

            l1 = plt.plot(bins, fitfunc(c_leastsq,bins), 'r--', linewidth=2)
            plt.title(f"Card{card_id}_Channel{chan}")
            plt.xlabel("Amplitude")
            plt.savefig(f"fig/20241127214613/{card_id}_{chan}_amplitude.pdf")
        plt.clf()

fig/20241127214613/7_0_amplitude.pdf, mean=66.1619777693021, sigma=23.017444892960636, mu=0.13068946780884397
fig/20241127214613/7_1_amplitude.pdf, mean=44.34344646623151, sigma=17.335953677324582, mu=0.19811442395264636
fig/20241127214613/7_2_amplitude.pdf, mean=64.9423296658529, sigma=22.68443673847734, mu=0.23575447157362664
fig/20241127214613/7_3_amplitude.pdf, mean=50.47126778877807, sigma=20.870757013928518, mu=0.10750840170404848
fig/20241127214613/7_4_amplitude.pdf, mean=59.53735593278823, sigma=22.16392062694509, mu=0.14037054767558582
fig/20241127214613/7_5_amplitude.pdf, mean=69.0399711261542, sigma=30.493209833113816, mu=0.028474391020566175
fig/20241127214613/7_6_amplitude.pdf, mean=52.77641158358971, sigma=21.314344722389848, mu=0.1063925732644841
fig/20241127214613/7_7_amplitude.pdf, mean=59.064190087121176, sigma=21.60341748759819, mu=0.16739434711626636
fig/20241127214613/7_8_amplitude.pdf, mean=51.10589017217399, sigma=20.772648926665948, mu=0.09449367916478153
fig/20

/tmp/katsui/ipykernel_2713811/362339076.py:5: RuntimeWarning: overflow encountered in exp
  fitfunc  = lambda p, x: p[0]*(exp(-0.5*((x-p[1])/p[2])**2)+p[5]/2/sqrt(2)*exp(-0.5*0.5*((x-2*p[1])/p[2])**2)+p[5]**2/2/3/sqrt(3)*exp(-0.5/3*((x-3*p[1])/p[2])**2))+p[3]*exp(-p[4]*x)


fig/20241127214613/7_16_amplitude.pdf, mean=56.1914642670334, sigma=21.345532111533974, mu=0.32680289939857243
fig/20241127214613/7_17_amplitude.pdf, mean=55.90764567478653, sigma=21.906775578845384, mu=0.24770576870076152
fig/20241127214613/7_18_amplitude.pdf, mean=60.0223698748506, sigma=20.64038132384141, mu=0.1649941377681985
fig/20241127214613/44_0_amplitude.pdf, mean=60.78302927828508, sigma=19.25610134659865, mu=0.33976622634295855
fig/20241127214613/44_1_amplitude.pdf, mean=62.92883155429495, sigma=20.84896579777517, mu=0.3169886050727875
fig/20241127214613/44_2_amplitude.pdf, mean=55.179270433356315, sigma=20.44160512787017, mu=0.2463795462343806
fig/20241127214613/44_3_amplitude.pdf, mean=54.7589221758035, sigma=24.630511987130276, mu=0.38333632437336485
fig/20241127214613/44_4_amplitude.pdf, mean=69.72747374085499, sigma=25.415363590699492, mu=0.23828944935088403
fig/20241127214613/44_5_amplitude.pdf, mean=60.24681113935046, sigma=23.092654088198227, mu=0.324708274694827
fig

/tmp/katsui/ipykernel_2713811/362339076.py:21: RuntimeWarning: Number of calls to function has reached maxfev = 1400.
  out   = leastsq( errfunc_leastsq, init, args=(bins, n))


fig/20241127214613/12_5_amplitude.pdf, mean=58.7446772103843, sigma=24.19238895521805, mu=0.05689692561611314


...

/tmp/katsui/ipykernel_2713811/362339076.py:5: RuntimeWarning: overflow encountered in exp
  fitfunc  = lambda p, x: p[0]*(exp(-0.5*((x-p[1])/p[2])**2)+p[5]/2/sqrt(2)*exp(-0.5*0.5*((x-2*p[1])/p[2])**2)+p[5]**2/2/3/sqrt(3)*exp(-0.5/3*((x-3*p[1])/p[2])**2))+p[3]*exp(-p[4]*x)
/tmp/katsui/ipykernel_2713811/362339076.py:21: RuntimeWarning: Number of calls to function has reached maxfev = 1400.
  out   = leastsq( errfunc_leastsq, init, args=(bins, n))


fig/20241127214613/85_0_amplitude.pdf, mean=90.77761125634471, sigma=29.652261819525346, mu=0.1420143256067454
fig/20241127214613/85_1_amplitude.pdf, mean=85.85628643179318, sigma=28.99823862840419, mu=0.200811420898122
fig/20241127214613/85_2_amplitude.pdf, mean=95.78721292333354, sigma=44.25319897764674, mu=-0.029053354855187046
fig/20241127214613/85_3_amplitude.pdf, mean=89.39510847437066, sigma=32.205158456329436, mu=0.15969800461547412
fig/20241127214613/85_4_amplitude.pdf, mean=83.44729917461792, sigma=30.774548615465914, mu=0.20833379193173437
fig/20241127214613/85_5_amplitude.pdf, mean=79.8094201822258, sigma=31.704560034419018, mu=0.20257771392473345
fig/20241127214613/85_6_amplitude.pdf, mean=84.8889063536785, sigma=34.3220798592628, mu=0.20520801240034253
fig/20241127214613/85_7_amplitude.pdf, mean=78.19818294147866, sigma=27.200579423366133, mu=0.23187237690644838
fig/20241127214613/85_8_amplitude.pdf, mean=98.7091103771774, sigma=30.95415559366541, mu=0.3893280844268142
fi

/tmp/katsui/ipykernel_2713811/362339076.py:21: RuntimeWarning: Number of calls to function has reached maxfev = 1400.
  out   = leastsq( errfunc_leastsq, init, args=(bins, n))


fig/20241127214613/89_17_amplitude.pdf, mean=92.87340120805932, sigma=27.181741455484076, mu=0.5868373872860418
fig/20241127214613/89_18_amplitude.pdf, mean=92.62550272986205, sigma=41.61070668337802, mu=-0.22704926329514127
fig/20241127214613/111_0_amplitude.pdf, mean=85.67807897029317, sigma=23.509248399602157, mu=-0.35546463030303604
fig/20241127214613/111_1_amplitude.pdf, mean=91.8956024590595, sigma=30.871162814781478, mu=0.13548458442876463
fig/20241127214613/111_2_amplitude.pdf, mean=99.94995731297684, sigma=32.50836495600691, mu=0.16394293708056198
fig/20241127214613/111_3_amplitude.pdf, mean=90.33223374764849, sigma=29.41019900334104, mu=0.10489389905962042
fig/20241127214613/111_4_amplitude.pdf, mean=90.36152769079219, sigma=24.713013259812605, mu=-0.48534000675426403
fig/20241127214613/111_5_amplitude.pdf, mean=89.09873671713929, sigma=30.67936559468919, mu=0.1261558546315856
fig/20241127214613/111_6_amplitude.pdf, mean=80.46693496147122, sigma=26.892124911349534, mu=0.10084

<Figure size 640x480 with 0 Axes>

In [6]:
plt.hist(gains,bins=40,range=[0,200])
plt.xlabel("1PE amplitude mean")
plt.savefig("fig/20241127214613/amplitude_gains.pdf")
plt.clf()

plt.hist(sigmas,bins=40,range=[0.2,0.6])
plt.xlabel("1PE amplitude sigma")
plt.savefig("fig/20241127214613/amplitude_sigmas.pdf")
plt.clf()

pmt_type = {}
pmt_type[0] = [3, 117]
#pmt_type[1] = [73, 117]
for (card_id, chan), value in pmt_dict.items():
    pmt_type[card_id] = [card_id,card_id]

for ptype, values in pmt_type.items():
    x=[]
    y=[]
    yerr=[]
    y_mu = []
    y_alpha = []
    for (card_id, chan), value in pmt_dict.items():
        #if gains[value]>0 and int(card_id)>=values[0] and int(card_id)<=values[1]:
        if int(card_id)>=values[0] and int(card_id)<=values[1]:
            x.append(int(card_id)+float(chan)/20.)
            y.append(gains[value])
            yerr.append(abs(gains[value]*sigmas[value]))
            y_mu.append(mu[value])
            y_alpha.append(alpha[value])
            #count += 1
    plt.errorbar(x,y,yerr=yerr,fmt='o')
    plt.grid(visible=True,which='both')
    plt.title(f"Gains and sigmas of all PMTs")
    plt.xlabel("CardID")
    plt.savefig(f"fig/20241127214613/gains_sigmas_type{ptype}.pdf")
    plt.clf()
    plt.scatter(x,y_mu,marker='o')
    plt.grid(visible=True,which='both')
    plt.title(f"Mean PE of all PMTs")
    plt.xlabel("CardID")
    plt.savefig(f"fig/20241127214613/pe_{ptype}.pdf")
    plt.clf()
    plt.scatter(x,y_alpha,marker='o')
    plt.grid(visible=True,which='both')
    plt.title(f"Mean alpha of all PMTs")
    plt.xlabel("CardID")
    plt.savefig(f"fig/20241127214613/alpha_{ptype}.pdf")
    plt.clf()

<Figure size 640x480 with 0 Axes>

In [23]:
plt.hist(digiQ,bins=40,range=[0,5])
plt.xlabel("Digi Q")
plt.savefig("fig/digiQ.pdf")
plt.clf()

plt.hist(digiT,bins=40,range=[-30.1,-22.1])
plt.xlabel("Digi T - Trigger time (ns)")
plt.savefig("fig/digiT.pdf")
plt.clf()

<Figure size 640x480 with 0 Axes>

In [ ]:
avg_fluc= []

for (card_id, chan), value in pmt_dict.items():
    print(f"fig/{card_id}_{chan}.pdf")
    x=[]
    y=[]
    yerr=[]
    yerr_frac=[]
    fluc = []
    #for i in range(pulses_nbins):
    for i in range(100):
        if len(pulses[value][i])>0:
            x.append(i*pulses_width)
            y.append(np.mean(pulses[value][i]))
            yerr.append(np.std(pulses[value][i]))
            yerr_frac.append(abs(np.std(pulses[value][i])/np.mean(pulses[value][i])))
            if i*pulses_width>=12 and i*pulses_width<=36:
                fluc.append(abs(np.std(pulses[value][i])/np.mean(pulses[value][i])))
    avg_fluc.append(np.mean(fluc))
    plt.errorbar(x,y,yerr=yerr)
    plt.plot(x,yerr_frac,'o-')
    plt.ylim(-0.15, 0.5)
    plt.xticks(np.linspace(0, 50, 6))
    plt.xticks(np.linspace(0, 50, 26), minor=True)
    #plt.xticks(np.linspace(0, 140, 8))
    plt.yticks(np.linspace(-0.1, 0.5, 7))
    plt.yticks(np.linspace(-0.15, 0.5, 14), minor=True)
    plt.grid(visible=True,which='both')
    plt.title(f"Card{card_id}_Channel{chan}")
    plt.xlabel("Time (ns)")
    plt.savefig(f"fig/{card_id}_{chan}.pdf")
    plt.clf()

plt.hist(avg_fluc,bins=100,range=[0,0.1])
plt.savefig("fig/avg_fluc.pdf")
plt.clf()

fig/12_0.pdf
fig/12_1.pdf
fig/12_2.pdf


fig/12_3.pdf
fig/12_4.pdf
fig/12_5.pdf
fig/12_6.pdf
fig/12_7.pdf
fig/12_8.pdf
fig/12_9.pdf
fig/12_10.pdf
fig/12_11.pdf
fig/12_12.pdf
fig/12_13.pdf
fig/12_14.pdf
fig/12_15.pdf
fig/12_16.pdf
fig/12_17.pdf
fig/12_18.pdf
fig/12_19.pdf
fig/111_0.pdf
fig/111_1.pdf


/afs/cern.ch/user/k/katsui/.local/lib/python3.9/site-packages/numpy/_core/fromnumeric.py:3596: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/afs/cern.ch/user/k/katsui/.local/lib/python3.9/site-packages/numpy/_core/_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


fig/111_2.pdf
fig/111_3.pdf
fig/111_4.pdf
fig/111_5.pdf
fig/111_6.pdf
fig/111_7.pdf
fig/111_8.pdf
fig/111_9.pdf
fig/111_10.pdf
fig/111_11.pdf
fig/111_12.pdf
fig/111_13.pdf
fig/111_14.pdf
fig/111_15.pdf
fig/111_16.pdf
fig/111_17.pdf
fig/111_18.pdf
fig/111_19.pdf


/afs/cern.ch/user/k/katsui/.local/lib/python3.9/site-packages/numpy/_core/fromnumeric.py:3596: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/afs/cern.ch/user/k/katsui/.local/lib/python3.9/site-packages/numpy/_core/_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


...

fig/26_1.pdf
fig/26_2.pdf
fig/26_3.pdf
fig/26_4.pdf
fig/26_5.pdf
fig/26_6.pdf
fig/26_7.pdf
fig/26_8.pdf
fig/26_9.pdf


/tmp/katsui/ipykernel_3768428/829466553.py:15: RuntimeWarning: invalid value encountered in scalar divide
  yerr_frac.append(abs(np.std(pulses[value][i])/np.mean(pulses[value][i])))


fig/26_10.pdf
fig/26_11.pdf
fig/26_12.pdf
fig/26_13.pdf
fig/26_14.pdf
fig/26_15.pdf
fig/26_16.pdf
fig/26_17.pdf
fig/26_18.pdf
fig/26_19.pdf


/afs/cern.ch/user/k/katsui/.local/lib/python3.9/site-packages/numpy/_core/fromnumeric.py:3596: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/afs/cern.ch/user/k/katsui/.local/lib/python3.9/site-packages/numpy/_core/_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


<Figure size 640x480 with 0 Axes>

In [ ]:
plt.a